In [15]:
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import UnexpectedAlertPresentException,NoSuchElementException, StaleElementReferenceException, ElementClickInterceptedException, ElementNotInteractableException, TimeoutException
import time
import requests
from bs4 import BeautifulSoup
import re
from selenium.webdriver.chrome.options import Options
chromeOptions= Options()
service = Service (executable_path="/Users/navneetyadav/Downloads/chromedriver-mac-arm64 2/chromedriver")
driver = webdriver.Chrome (service=service,options=chromeOptions)


In [16]:
df = pd.read_excel('/Users/navneetyadav/Downloads/Dependo_ItemList01.xlsx')
df

,S.no,Product,Quantity,As per Amazon Price,Location
0,1,"AGARO BLDC Professional Hair Dryer, Brushless ...",1.0,7759,HYD
1,2,Marshall Emberton II Wireless Bluetooth Portab...,1.0,17499,HYD
2,3,Marshall Emberton II Wireless Bluetooth Portab...,1.0,19999,HYD
3,4,MI Portable Bluetooth Speaker with 16W Hi-Qual...,2.0,4598,HYD
4,5,Fitbit Versa 4 Fitness Watch (Black / Graphite...,1.0,20499,HYD
...,...,...,...,...,...
399,400,PIGON GAS CHULHA,NaN,4000,Ambala_2
400,401,STIBIRD -GRINDER,NaN,1200,Ambala_2
401,402,LOCKER,NaN,5000,Ambala_2
402,403,CHOCKRY SET MAHARAJ MALMINE,NaN,1700,Ambala_2


In [17]:
## adding empty columns 
df["Amazon Website URL"]= None
df["Product Title"]=None
df["About the product"]=None
df["Product Detail"]=None
df["Product Detail2"]=None
df

,S.no,Product,Quantity,As per Amazon Price,Location,Amazon Website URL,Product Title,About the product,Product Detail,Product Detail2
0,1,"AGARO BLDC Professional Hair Dryer, Brushless ...",1.0,7759,HYD,None,None,None,None,None
1,2,Marshall Emberton II Wireless Bluetooth Portab...,1.0,17499,HYD,None,None,None,None,None
2,3,Marshall Emberton II Wireless Bluetooth Portab...,1.0,19999,HYD,None,None,None,None,None
3,4,MI Portable Bluetooth Speaker with 16W Hi-Qual...,2.0,4598,HYD,None,None,None,None,None
4,5,Fitbit Versa 4 Fitness Watch (Black / Graphite...,1.0,20499,HYD,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
399,400,PIGON GAS CHULHA,NaN,4000,Ambala_2,None,None,None,None,None
400,401,STIBIRD -GRINDER,NaN,1200,Ambala_2,None,None,None,None,None
401,402,LOCKER,NaN,5000,Ambala_2,None,None,None,None,None
402,403,CHOCKRY SET MAHARAJ MALMINE,NaN,1700,Ambala_2,None,None,None,None,None


In [18]:
#for i in range(len(df)):
for i in range(71,len(df)):
    driver.get('https://www.google.com/')
    
    prod_name=df.iloc[i]['Product ']
    search = prod_name

    print(i," ",prod_name)
    
    # Wait for the search input field to be present and visible
    search_input = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.NAME, 'q')))
    # Enter the search query and submit
    search_input.send_keys(search + Keys.ENTER)
    time.sleep(2)
    intial_Google_url=driver.current_url

    # for finding amazon website for the product
    try:
        wait = WebDriverWait(driver, 5) # intially 60 (driver,60)
        amazon_click_button = wait.until(EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, 'Amazon')))
        time.sleep(2)
        amazon_click_button.click() 

        print("------------------------------------Reached websited---------------------------------------------") 

        print("------------------------------------"+driver.current_url+"------------------------------------")
        current_url=driver.current_url
        df.loc[i,"Amazon Website URL"]= driver.current_url

        if current_url != intial_Google_url:
            
            # for adding title
            try:
                prod_title= wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#title.a-size-large.a-spacing-none')))
                df.loc[i,"Product Title"]= prod_title.text
            except (UnexpectedAlertPresentException,TimeoutException,StaleElementReferenceException, NoSuchElementException,ElementClickInterceptedException,ElementNotInteractableException): 
                df.loc[i,"Product Title"]=  ""
    
            print("------------------------------------Product Title------------------------------------")
            print(df.iloc[i]["Product Title"])
            
            # About the product 
            try:
                about_the_prod =ul_element = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "ul.a-unordered-list.a-vertical.a-spacing-mini")))
                df.loc[i,"About the product"]= about_the_prod.text
            except (UnexpectedAlertPresentException,TimeoutException,StaleElementReferenceException, NoSuchElementException,ElementClickInterceptedException,ElementNotInteractableException): 
                df.loc[i,"About the product"] = ""
    
            print("------------------------------------About the product------------------------------------")
            print(df.iloc[i]["About the product"])
            
            # for product Details
            try:
                prod_details= wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#prodDetails.a-section")))
                df.loc[i,"Product Detail"]= prod_details.text
            except (UnexpectedAlertPresentException,TimeoutException,StaleElementReferenceException, NoSuchElementException,ElementClickInterceptedException,ElementNotInteractableException): 
                df.loc[i,"Product Detail"] = ""
    
            print("------------------------------------Product Detail------------------------------------")
            print(df.iloc[i]["Product Detail"])
            
            # for more product Details
            try:
                prod_details2= wait.until(EC.visibility_of_element_located((By.XPATH,"//div[@id='detailBullets_feature_div']")))
                df.loc[i,"Product Detail2"]= prod_details2.text
            except (UnexpectedAlertPresentException,TimeoutException,StaleElementReferenceException, NoSuchElementException,ElementClickInterceptedException,ElementNotInteractableException): 
                df.loc[i,"Product Detail2"] = ""
    
            print("------------------------------------Product Detail2------------------------------------")
            print(df.iloc[i]["Product Detail2"])
    
            time.sleep(2)

        else:
            time.sleep(2)
            continue
    except (UnexpectedAlertPresentException,TimeoutException,StaleElementReferenceException, NoSuchElementException,ElementClickInterceptedException,ElementNotInteractableException): 
        time.sleep(2)
        continue
        
        




71   WOODEN TEMPLE
------------------------------------Reached websited---------------------------------------------
------------------------------------https://www.amazon.in/Home-Temples-Over-%E2%82%B93-000/s?rh=n%3A5744070031%2Cp_36%3A3444814031------------------------------------
------------------------------------Product Title------------------------------------

------------------------------------About the product------------------------------------

------------------------------------Product Detail------------------------------------

------------------------------------Product Detail2------------------------------------

72   4K WIFI ACTION CAMERA
------------------------------------Reached websited---------------------------------------------
------------------------------------https://www.amazon.in/s?k=4k+wifi+action+camera&adgrpid=61797731987&ext_vrnc=hi&gclid=EAIaIQobChMIgu7r-_L_hQMVlxiDAx2sdgewEAAYASAAEgKpkfD_BwE&hvadid=590641643178&hvdev=c&hvlocphy=9301792&hvnetw=g&hvqm

In [19]:
df.to_excel("/Users/navneetyadav/Downloads/Output2.xlsx", index=False)

In [21]:
driver.close()